In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import cvzone

# Load a model
model = YOLO("yolov8n-pose.pt") 

cap = cv2.VideoCapture('sw.mp4')

# Define the codec and create a VideoWriter object to save the output
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can change the codec as needed
output_width = 1200
output_height = 680
out = cv2.VideoWriter('posees.mp4', fourcc, 30, (output_width, output_height))

# Define a function to calculate the angle between three keypoints
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

while True:
    ret, frame = cap.read()
    
    try:
        frame = cv2.resize(frame, (output_width, output_height))
    except:
        pass

    if not ret:
        break

    results = model.predict(frame, save=True)

    # Get the bounding box information in xyxy format
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)

    statuses = []

    # Get the keypoints data for all detected persons
    keypoints_data = results[0].keypoints.data

    # Iterate through the detected persons
    for i, keypoints in enumerate(keypoints_data):
        # Ensure keypoints are detected
        if keypoints.shape[0] > 0:
            angle = calculate_angle(keypoints[11][:2], keypoints[13][:2], keypoints[15][:2])  # Angle between head, hips, and knees
            print(f"Person {i + 1} is {'Sitting' if angle is not None and angle < 110 else 'Standing'} (Angle: {angle:.2f} degrees)")
            statuses.append('Sitting' if angle is not None and angle < 110 else 'Standing')

    # Draw bounding boxes and statuses on the frame
    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cvzone.putTextRect(
            frame, f"{statuses[i]}", (x1, y2 - 10),  # Image and starting position of the rectangle
            scale=3, thickness=3,  # Font scale and thickness
            colorT=(255, 255, 255), colorR=(255, 0, 255),  # Text color and Rectangle color
            font=cv2.FONT_HERSHEY_PLAIN,  # Font type
            offset=10,  # Offset of text inside the rectangle
            border=0, colorB=(0, 255, 0)  # Border thickness and color
        )

    # Write the frame to the output video file
    out.write(frame)

    # Show the pose detection on the frame
    detection = results[0].plot()

    # Display the resulting image with pose detection and statuses
    cv2.imshow('YOLOv8 Pose Detection', detection)

    # Exit the program if the user presses the 'q' key
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture device, close the output video file, and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()
